# 3. Feature Enginerring

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os 
import sys
sys.path.append(os.path.join(os.path.abspath('..')))

In [3]:
import src.data_loading as dl
import scripts.feature_enigneering as fe

In [4]:
output_data = '../data/processed'
df = pd.read_csv(output_data + '/processed_data.csv')
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15T02:44:21Z,2,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15T03:34:21Z,2,0


1. Create Aggregate Features

In [5]:
# Total transaction amount 
df['TotalTransactionAmount'] = df.groupby('CustomerId')['Value'].transform('sum')

In [6]:
# df['TotalT'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().sum()))

In [7]:
# Average Transaction Amount
df['AverageTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().mean()))

In [8]:
# df['ava'] = df.groupby('CustomerId')['Value'].transform('mean')

In [9]:
# Transaction Count
df['TransationCount'] = df.groupby('CustomerId')['TransactionId'].transform('size')

Note that the std function will calculate the population standard deviation by default. So i want to calculate the sample standard deviation, so i can use the ddof parameter:

In [10]:
# Standard Deviation of Transaction Amounts
df['StdTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().std(ddof=1)))

In [11]:
# df['StdDevTransaction'] = df.groupby('CustomerId')['Value'].transform('std')

In [12]:
#categorical and numerical columns into variables
num_col = df.select_dtypes(include=np.number).columns
cat_col = df.select_dtypes(exclude=np.number).columns

## 2. Extract Features

In [13]:
# Apply the function
fe.extract_features(df, ['TransactionStartTime'])

In [14]:
pd.set_option('display.max_columns', None)
df.sample(6)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TransationCount,StdTransactionAmount,TransactionStartTime_Month,TransactionStartTime_Week,TransactionStartTime_Day,TransactionStartTime_Dayofweek,TransactionStartTime_Dayofyear,TransactionStartTime_Is_month_end,TransactionStartTime_Is_month_start,TransactionStartTime_Is_quarter_end,TransactionStartTime_Is_quarter_start,TransactionStartTime_Is_year_end,TransactionStartTime_Is_year_start,TransactionStartTime_Hour,TransactionStartTime_Minute,TransactionStartTime_Second,TransactionStartTime_Elapsed
91089,109376,45683,3432,841,3867,UGX,256,6,3,airtime,3,1000.0,1000,2019-02-08 19:15:49,2,0,940979,3362.142857,273,10761.786054,2,6,8,4,39,False,False,False,False,False,False,19,15,49,1549653349
38597,89317,93589,2058,3579,2461,UGX,256,6,4,airtime,3,1000.0,1000,2018-12-23 21:10:26,2,0,5550,792.857143,7,376.860464,12,51,23,6,357,False,False,False,False,False,False,21,10,26,1545599426
86234,43141,91893,3952,1324,4401,UGX,256,6,10,airtime,3,1300.0,1300,2019-02-05 15:43:37,2,0,70000,2333.333333,30,5420.674022,2,6,5,1,36,False,False,False,False,False,False,15,43,37,1549381417
33840,35528,51643,4421,4038,4878,UGX,256,6,10,airtime,3,30000.0,30000,2018-12-20 14:48:04,2,0,50912840,248355.317073,205,481165.515118,12,51,20,3,354,False,False,False,False,False,False,14,48,4,1545317284
69198,48754,81097,4841,3829,4430,UGX,256,4,6,financial_services,2,-10.0,10,2019-01-23 13:49:35,2,0,28310,1665.294118,17,4759.867209,1,4,23,2,23,False,False,False,False,False,False,13,49,35,1548251375
82932,131423,120090,2659,3327,3078,UGX,256,6,10,airtime,3,1000.0,1000,2019-02-02 06:04:44,2,0,2541260,4431.867365,573,22519.891852,2,5,2,5,33,False,False,False,False,False,False,6,4,44,1549087484


In [15]:
# Boolean to int 
df[df.select_dtypes(include='bool').columns] = df.select_dtypes(include='bool').astype(int)

## Encode Categorical Variables

In [ ]:
# Frequency Encoding
fe.encode_FE(df, ['CustomerId', 'TransactionId'])

In [ ]:
# Label Encoding
fe.encode_LE(df, ['ProductCategory', 'ProductId', 'SubscriptionId', 'InvestorId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ChannelId', 'PricingStrategy'])

In [ ]:
# Group Aggregation
fe.encode_AG(df, 'CustomerId', ['Value'], ['mean','min','max','std'])

In [ ]:
# Combine Features
fe.encode_CB(df, 'ProductId', 'ProductCategory')

In [ ]:
# Group Aggregation NUnique
fe.encode_AG2(df, 'CustomerId', ['TransactionId'])